In [2]:
def combined_gaussian_centers(gaussian_centers_coronal, contrast_intensities_coronal,
                              gaussian_centers_sagittal=np.array([]), contrast_intensities_sagittal=np.array([]),
                              gaussian_centers_axial=np.array([]), contrast_intensities_axial=np.array([]),
                              maxInd=np.array([]), savepath=None):
    num_patterns = 0
    coronal_exists = 0
    sagittal_exists = 0
    axial_exists = 0
    
    if gaussian_centers_coronal.any():
        num_patterns = len(gaussian_centers_coronal)
        coronal_exists = 1
        if contrast_intensities_coronal.any():
            coronal_weights = contrast_intensities_coronal
        else:
            coronal_weights = np.ones((num_patterns,))
    else: 
        coronal_weights = 0
        
    if gaussian_centers_sagittal.any():
        num_patterns = len(gaussian_centers_sagittal)
        sagittal_exists = 1
        if contrast_intensities_sagittal.any():
            sagittal_weights = contrast_intensities_sagittal
        else:
            sagittal_weights = np.ones((num_patterns,))
    else:
        sagittal_weights = 0

    if gaussian_centers_axial.any():
        num_patterns = len(gaussian_centers_axial)
        axial_exists = 1 
        if contrast_intensities_axial.any():
            axial_weights = contrast_intensities_axial
        else:
            axial_weights = np.ones((num_patterns,))
    else:
        axial_weights = 0

    gaussian_centers=np.zeros((num_patterns, 3))


    
    if gaussian_centers_coronal.any():
        gaussian_centers[:,0] = gaussian_centers_coronal[:,1]*coronal_weights
        gaussian_centers[:,1] = gaussian_centers_coronal[:,0]*coronal_weights

    if gaussian_centers_sagittal.any():
        gaussian_centers[:,2] = gaussian_centers_sagittal[:,0]*sagittal_weights
        gaussian_centers[:,1] = (gaussian_centers[:,1] + gaussian_centers_sagittal[:,1]*sagittal_weights)
    if gaussian_centers_axial.any():
        gaussian_centers[:,2] = (gaussian_centers[:,2] + gaussian_centers_axial[:,0]*axial_weights)
        gaussian_centers[:,0] = (gaussian_centers[:,0] + gaussian_centers_axial[:,1]*axial_weights)

    if gaussian_centers[:,0].any():
        gaussian_centers[:,0] = gaussian_centers[:,0]/(axial_weights + coronal_weights)
    if gaussian_centers[:,1].any():
        gaussian_centers[:,1] = gaussian_centers[:,1]/(sagittal_weights + coronal_weights)
    if gaussian_centers[:,2].any():
        gaussian_centers[:,2] = gaussian_centers[:,2]/(sagittal_weights + axial_weights)
    
    # if gaussian_centers_sagittal.any():
    #     for i in range(len(gaussian_centers_coronal)):
    #         pc=gaussian_centers_coronal[i]
    #         ps=gaussian_centers_sagittal[i]
    #         if contrast_intensities_coronal.any():
    #             int_c = contrast_intensities_coronal[i]
    #             int_s = contrast_intensities_sagittal[i]
    #         else:
    #             int_c = 1
    #             int_s = 1
    #         int_sum = int_c + int_s

    #         # Weighted average of gaussian centers
    #         gaussian_centers[i,:]=np.array([pc[1], pc[0]*int_c/int_sum + ps[1]*int_s/int_sum, ps[0]])
    #         # gaussian_centers[i,:]=np.array([pc[1],  np.mean([pc[0], ps[1]]), ps[0]])
            
    # elif gaussian_centers_axial.any():
    #     for i in range(len(gaussian_centers_coronal)):
    #         pc=gaussian_centers_coronal[i]
    #         pa=gaussian_centers_axial[i]
    
    # #         gaussian_centers[i,:]=np.array([pc[1],  np.mean([pc[0], ps[1]]), ps[0]])
    #         gaussian_centers[i,:]=np.array([np.mean([pc[1], pa[1]]), pc[0], pa[0]])
            
    # elif maxInd.any():
    #     for i in range(len(gaussian_centers_coronal)):
    #         pc=gaussian_centers_coronal[i]
    #         gaussian_centers[i,:]=np.array([pc[1],  pc[0], maxInd[2]])

    if savepath:
        filename = "gaussian_centers_3D.npy"
        np.save(os.path.join(savepath, filename), gaussian_centers)
        
    return gaussian_centers

NameError: name 'np' is not defined

In [ ]:
def get_cy_trapezoid(b,a,h):
    # Trapezoid centroid calculation
    #        a
    #     ________
    #    /|       \
    #   / |        \
    #  /  |h  .C    \
    # /___|__________\
    #.        b 

    cy=(b+2*a)*h/(3*(b+a))
    
    return cy

def get_cy_triangle(h):
    return h/3

def get_dist_to_deepest_ch(mrid_dict, offset=11):
    """
    Returns the distance from the deepest channel to the CoM of the deepest pattern
    """
    deepest_pattern = mrid_dict["dimensions"][-1,:]
    b,a,h=deepest_pattern

    if a>0:
        cy=get_cy_trapezoid(b,a,h)
    else:
        cy=get_cy_triangle(h)

    dist = h-cy+offset
    return dist

In [ ]:
def get_centomass(mrid_wafer_dimensions, intersegment_lengths):
    """
    mrid_wafer_dimensions: dimension vector of each pattern in mrid (num_patterns-by-3), dorsal-to-ventral, each row represents b,a,h; a=0 for triangular patterns
    intersegment_lengths: inter-pattern(segment) lengths starting from dorsal to ventral patterns
    """
    num_patterns=len(mrid_wafer_dimensions)
    c2c=np.zeros((num_patterns-1,))
    
    bundle_l=0
    ticks=[bundle_l]
    for i in range(num_patterns-1):
        b,a,h=mrid_wafer_dimensions[i,:]
        b2,a2,h2=mrid_wafer_dimensions[i+1,:]

        if a>0:
            cy=get_cy_trapezoid(b,a,h)
        else:
            cy=get_cy_triangle(h)
            
        bundle_l=bundle_l+cy
        ticks.append(bundle_l)

        distCy=h-cy
        bundle_l=bundle_l+distCy
        ticks.append(bundle_l)
        
        bundle_l=bundle_l+intersegment_lengths[i]
        ticks.append(bundle_l)
        
        if a2>0:  
            cy2=get_cy_trapezoid(b2,a2,h2)
#             distCy2=cy2
        else:
            cy2=get_cy_triangle(h2)
#             distCy2=cy2
        c2c[i]=distCy + intersegment_lengths[i] + cy2
    
    b,a,h=mrid_wafer_dimensions[-1,:]
    if a>0:
        cy=get_cy_trapezoid(b,a,h)
    else:
        cy=get_cy_triangle(h)

    bundle_l=bundle_l+cy
    ticks.append(bundle_l)

    distCy=h-cy
    bundle_l=bundle_l+distCy
    ticks.append(bundle_l)
    
    return c2c, ticks

def get_pattern_ratios(wafer_ticks):
    num_patterns=int(len(wafer_ticks)/3)
    ratios=np.zeros((num_patterns,))
    for pattern in range(num_patterns):
        p1,p2,p3=wafer_ticks[3*pattern:3*(pattern+1)]
        l=p3-p1
        c_y=p2-p1
        ratio=c_y/l
        ratios[pattern]=ratio
    
    return ratios

In [ ]:
def gaus(x,a,x0,sigma):
    return a*np.exp(-(x-x0)**2/(2*sigma**2))

def get_maxproj(heatmap_resampled, segmentation_resampled, roi_name, labelsdf, orientation):
    """
    Returns the maximum projection of the contrast heatmap volume in given orientation (i.e. axis)
    """
    if orientation=="coronal":
        axis=2
    elif orientation=="sagittal":
        axis=0
    elif orientation=="axial":
        axis=1

    
    # labelsdf=read_labels(os.path.join(anatpath, "labels.txt"))    
    roi_areas=labelsdf["Labels"][labelsdf["Anatomical Regions"].str.contains(roi_name)]

    ind = np.unravel_index(np.argmax(heatmap_resampled, axis=None), heatmap_resampled.shape)

    template = np.max(heatmap_resampled, axis=axis)
    # segmentationResampledMax = np.max(segmentation_resampled, axis=axis)

    # heatmaps = np.zeros((len(roi_areas), np.shape(heatmapResampledMax)[0], np.shape(heatmapResampledMax)[1]))
    heatmaps = np.zeros((len(roi_areas), np.shape(template)[0], np.shape(template)[1]))
    
    for i, roi in enumerate(roi_areas):
        roiHeatmap = heatmap_resampled*(segmentation_resampled==roi*1)
        heatmaps[i, :, :] = np.max(roiHeatmap, axis=axis)
        # heatmaps[i, :, :] = heatmapResampledMax*(segmentationResampledMax==roi*1)
        
    return heatmaps, ind 

def get_mrid_segment_length(amp, sigma, baseline_sigma, px_size=25):
    std=baseline_sigma
    mu=50
    l=100
    x = np.linspace(0, l, l*10)
    pdf = gaus(x,amp,mu,sigma)
    plt.plot(x,pdf)
    plt.hlines(2*std,0,100)
    start=np.min(x[np.where(pdf>2*std)])
    stop=np.max(x[np.where(pdf>2*std)])
#     print(start)
#     print(stop)
    print("Segment Length (um): "+str((stop-start)*px_size))
    
    return (stop-start)*px_size

In [4]:
# def gen_barcode_mrid(patternLengths, c2c_dists, barcode_length=4500, barcode_start=1000):
    
#     barcode = np.ones((1000,barcode_length))
#     l=barcode_start
#     ticks=[l]
        
#     for i, bar in enumerate(patternLengths):
#         if l==barcode_start:
#             barInt=np.round(bar).astype(int)
#             barcode[:, l:l+barInt]=0
#             ticks.append(l+barInt/2)
#             ticks.append(l+barInt)
#             l=l+np.round(bar/2).astype(int)+np.round(c2c_dists[i]).astype(int)
#         else:
#             barInt=np.round(bar/2).astype(int)
#             barcode[:, l-barInt:l+barInt]=0
#             ticks.append(int(l-barInt))
#             ticks.append(l)
#             ticks.append(l+barInt)
#             if i<len(c2c_dists):
#                 l=l+np.round(c2c_dists[i]).astype(int)
                
#     ticks=np.array(ticks)
#     tickLabels=(ticks-barcode_start).astype(int)
    
#     return barcode, ticks, tickLabels

# def gen_barcode_wafer(patternPoints, barcode_start=1000, barcode_length=4500):
#     num_patterns=int(len(patternPoints)/3)
#     barcode = np.ones((1000,barcode_length))
#     l=barcode_start

#     for pattern in range(num_patterns):
#         p1,p2,p3=patternPoints[3*pattern:3*(pattern+1)]
#         barcode[:, int(l+p1):int(l+p3+1)]=0

#     return barcode

# def equalize_barcodes(barcode1, barcode2):
#     if np.shape(barcode1)[1]>np.shape(barcode2)[1]:
#         diff=np.shape(barcode1)[1]-np.shape(barcode2)[1]
#         padd=np.ones((np.shape(barcode2)[0], diff))
#         barcode2=np.concatenate((padd, barcode2), axis=1)
#     elif np.shape(barcode1)[1]<np.shape(barcode2)[1]:
#         diff=np.shape(barcode2)[1]-np.shape(barcode1)[1]
#         padd=np.ones((np.shape(barcode1)[0], diff))
#         barcode1=np.concatenate((padd, barcode1), axis=1)
    
#     return barcode1, barcode2
# def barcode_cc(barcode_fix, barcode_mvng):
# #     barcode_fix, barcode_mvng = equalize_barcodes(barcode_fix, barcode_mvng)
#     stride=1
#     cc=[]
#     barcode_fix=1-barcode_fix
#     barcode_fix=barcode_fix[0,:]
# #     print("zero padding: "+str(np.shape(barcode_mvng)[-1]))
#     barcode_fix=np.append(np.zeros((np.shape(barcode_mvng)[-1])), barcode_fix)
#     barcode_fix=np.append(barcode_fix, np.zeros((np.shape(barcode_mvng)[-1])))
    
#     barcode_mvng=1-barcode_mvng
#     barcode_mvng=barcode_mvng[0,:]
#     for i in range(len(barcode_fix)-len(barcode_mvng)):
#         corr=np.multiply(barcode_fix[stride*i:stride*i+len(barcode_mvng)], barcode_mvng)
#         cc.append(np.sum(corr))    
    
#     return cc

In [1]:
def find_gaussian_centers(heatmaps, img, px_size, coronal=True, savepath=None, filename=None):
    """
    heatmaps: relaxation heatmaps generated from raw MGE images, and resampled to space of interest. num_patterns-by-img_rows-by-img_colmns
    img: img to be plotted as base to the analysis
    px_size: size of each pixel
    """
    print("Shape of the heatmaps data: ")
    print(np.shape(heatmaps))
    num_rows = np.shape(heatmaps)[1]
    num_clms = np.shape(heatmaps)[2]

    y=np.linspace(0,num_rows-1,num_rows)
    x=np.linspace(0,num_clms-1,num_clms)
    x, y = np.meshgrid(x, y)

    # # create a colormap object
    cmap = plt.cm.get_cmap("jet").copy()
    cmap.set_under('white',  alpha=0)

    # # set colourbar map
    cmap_args = dict(cmap=cmap, vmin=1)

    fig, ax = plt.subplots(1, 1)
    ax.imshow(img, cmap='gray')
    ax.imshow(np.sum(heatmaps, axis=0), **cmap_args, alpha=0.75)
    
    numRois, _, _ = np.shape(heatmaps)
    t2strength=np.zeros((numRois,))
    gaussAmp=np.zeros((numRois,))
    gaussSig=np.zeros((numRois,))
    gaussian_centers=[]

    for k in range(numRois):
        i,j=np.unravel_index(np.argmax(heatmaps[k,:,:]), (num_rows, num_clms))
        z=heatmaps[k,:,:]
        initial_guess = (45,j,i,2,2,0,0)
        data_fitted, popt, pcov=fit_2d_gaussian((x, y), z, initial_guess)
        gaussian_centers.append(popt[1:3])
        gauss_x=np.round(popt[1]).astype(int)
        gauss_y=np.round(popt[2]).astype(int)
        print("Fitted Gaussian x: ")
        print(gauss_x)
        print("Fitted Gaussian y: ")
        print(gauss_y)
        t2strength[k]=z[gauss_y, gauss_x]
        gaussAmp[k]=popt[0]
        thetaDeg = (popt[5]%(2*np.pi))*180/np.pi
        print("theta(radians): "+str(popt[5]))
        print("theta: "+str(thetaDeg))
        if coronal:
            if (thetaDeg > 45 and thetaDeg < 135) or (thetaDeg > 225 and thetaDeg < 315):
                gaussSig[k] = popt[4]
            else:
                gaussSig[k] = popt[3]
        else:
            if (thetaDeg > 45 and thetaDeg < 135) or (thetaDeg > 225 and thetaDeg < 315):
                gaussSig[k] = popt[3]
            else:
                gaussSig[k] = popt[4]

        print(popt[1:3])
        print("sigma: "+str(popt[3:5]))
        print("Legth_sigma: "+str(gaussSig[k]))
        print("len: " + str(gaussSig[k]*2*px_size))
        
        print("amp: "+str(popt[0]))
        ax.contour(x, y, data_fitted.reshape((num_rows, num_clms)), 4, linewidths=1, colors='black', alpha=0.15)

    gaussian_centers=np.array(gaussian_centers)
    
    if savepath and filename:
        plt.savefig(os.path.join(savepath, filename), dpi=1000)
    plt.show()
    
    return gaussian_centers, gaussAmp, gaussSig, popt